In [ ]:
import pandas as pd

# Baca file Excel
df = pd.read_excel('data.xlsx')

# Transformasi data menjadi format lebar (wide format)
pivot_data = df.pivot_table(
    index=['Menu'],
    columns=['Bulan', 'Tahun'],
    values='Terjual',
    aggfunc='sum',
    fill_value=0
)

# Urutkan bulan dan tahun secara berurutan
pivot_data = pivot_data.sort_index(axis=1, level=[1, 0])

# Tambahkan kolom 'Jumlah' untuk total penjualan
pivot_data['Jumlah'] = pivot_data.sum(axis=1)

# Hitung persentil 50 dan 75 untuk penentuan kategori
percentile_50 = pivot_data['Jumlah'].quantile(0.50)
percentile_75 = pivot_data['Jumlah'].quantile(0.75)

# Fungsi untuk menentukan kategori
def categorize_sales(value):
    if value > percentile_75:
        return 'Paling Laris'
    elif value > percentile_50:
        return 'Laris'
    else:
        return 'Kurang Laris'

# Tambahkan kolom 'Kategori'
pivot_data['Kategori'] = pivot_data['Jumlah'].apply(categorize_sales)

# Definisikan bulan untuk data latih dan uji
train_months = [
    ('September', 2022), ('Oktober', 2022), ('November', 2022), ('Desember', 2022),
    ('Januari', 2023), ('Februari', 2023), ('Maret', 2023),
    ('September', 2023), ('Oktober', 2023), ('November', 2023), ('Desember', 2023),
    ('Januari', 2024), ('Februari', 2024), ('Maret', 2024)
]

test_months = [
    ('April', 2024), ('Mei', 2024), ('Juni', 2024),
    ('Juli', 2024), ('Agustus', 2024)
]

# Ekstrak data latih dan uji
train_data = pivot_data[train_months + [('Kategori', '')]].copy()
test_data = pivot_data[test_months].copy()

# Reset indeks untuk kejelasan
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

# Ratakan (flatten) kolom MultiIndex
train_data.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in train_data.columns]
test_data.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in test_data.columns]

# Simpan data latih dan uji ke file Excel terpisah
train_file = "train_data.xlsx"
test_file = "test_data.xlsx"

train_data.to_excel(train_file, index=False, sheet_name='Training Data')
test_data.to_excel(test_file, index=False, sheet_name='Test Data')

print(f"Data latih berhasil diekspor ke: {train_file}")
print(f"Data uji berhasil diekspor ke: {test_file}")


Data latih berhasil diekspor ke: train_data.xlsx
Data uji berhasil diekspor ke: test_data.xlsx


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Load Data
train_data = pd.read_excel("train_data.xlsx")
test_data = pd.read_excel("test_data.xlsx")

# 2. Preprocessing: Pisahkan fitur dan target
X_train = train_data.drop(columns=['Menu_', 'Kategori_'], errors='ignore')  # Fitur
y_train = train_data['Kategori_']  # Target

# 3. Pastikan fitur data uji sesuai dengan fitur data latih
X_test = test_data.drop(columns=['Menu_'], errors='ignore')
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)  # Samakan kolom dan isi yang hilang dengan 0

# 4. Inisialisasi dan Latih Model KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# 5. Prediksi Kategori untuk Data Uji
y_test_pred = knn.predict(X_test)
test_data['Kategori_Prediksi'] = y_test_pred

# 6. Evaluasi Model dengan Data Latih
y_train_pred = knn.predict(X_train)
print("Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:\n", classification_report(y_train, y_train_pred))
print(f"Accuracy: {accuracy_score(y_train, y_train_pred) * 100:.2f}%")

# 7. Simpan Hasil Prediksi
test_data_file = "test_data_with_predictions.xlsx"
test_data.to_excel(test_data_file, index=False, sheet_name='Test Data with Predictions')

print(f"Hasil prediksi disimpan di: {test_data_file}")


Confusion Matrix:
 [[12  1  0]
 [ 3  3  0]
 [ 0  1  6]]

Classification Report:
               precision    recall  f1-score   support

Kurang Laris       0.80      0.92      0.86        13
       Laris       0.60      0.50      0.55         6
Paling Laris       1.00      0.86      0.92         7

    accuracy                           0.81        26
   macro avg       0.80      0.76      0.78        26
weighted avg       0.81      0.81      0.80        26

Accuracy: 80.77%
Hasil prediksi disimpan di: test_data_with_predictions.xlsx


Interpretasi:

Kurang Laris (Baris 1):
*   12 data Kurang Laris diprediksi benar.
*   1 data Kurang Laris salah diprediksi sebagai Laris.

Laris (Baris 2):
*   3 data Laris salah diprediksi sebagai Kurang Laris.
*   3 data Laris diprediksi dengan benar.

Paling Laris (Baris 3):
*   6 data Paling Laris diprediksi dengan benar.
*   1 data Paling Laris salah diprediksi sebagai Laris

Penjelasan Metrik:

Precision:
*   Proporsi prediksi benar dari total prediksi untuk kelas tertentu.
*   Contoh: Precision Kurang Laris = 0.80 → Dari semua prediksi "Kurang Laris", 80% benar.

Recall:
*   Proporsi data benar yang berhasil diprediksi dengan tepat.
*   Contoh: Recall Kurang Laris = 0.92 → Dari semua data Kurang Laris, 92% berhasil diprediksi benar.

F1-Score:
*   Rata-rata harmonis antara precision dan recall, berguna jika ada ketidakseimbangan kelas.
*   Contoh: F1-score Kurang Laris = 0.86 → Model bekerja cukup baik untuk kelas ini.

Support:
*   Jumlah data untuk setiap kategori di dataset.
*   Contoh: Ada 13 data Kurang Laris, 6 data Laris, dan 7 data Paling Laris.





